# News Classification Based on Their Headlines

### This notebook outlines the process for classifying news based on their headlines using text mining and NLP techniques.

# Import necessary libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


# Load the dataset

In [2]:
# Replace 'path_to_dataset.csv' with the actual file path
dataframe = pd.read_csv('uci-news-aggregator.csv', encoding="utf8", usecols=['TITLE', 'CATEGORY'])
dataframe.columns

Index(['TITLE', 'CATEGORY'], dtype='object')

In [3]:
# Display the dataset
dataframe

,TITLE,CATEGORY
0,"Fed official says weak data caused by weather,...",b
1,Fed's Charles Plosser sees high bar for change...,b
2,US open: Stocks fall after Fed official hints ...,b
3,"Fed risks falling 'behind the curve', Charles ...",b
4,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,b
...,...,...
422414,Surgeons to remove 4-year-old's rib to rebuild...,m
422415,Boy to have surgery on esophagus after battery...,m
422416,Child who swallowed battery to have reconstruc...,m
422417,Phoenix boy undergoes surgery to repair throat...,m


# Data preprocessing

In [4]:
# Preprocessing
#check for missing data
if(any(dataframe.isnull().any())):
    print('Missing Data\n')
    print(dataframe.isnull().sum())
else:
    print('NO missing data')
    

NO missing data


In [5]:
# check for duplicate
if(any(dataframe.duplicated())==True):
    print('Duplicate rows found')
    print('Number of duplicate rows= ', dataframe[dataframe.duplicated()].shape[0])
    dataframe.drop_duplicates(inplace=True,keep='first')
    dataframe.reset_index(inplace=True,drop=True)
    print('Dropping duplicates\n')
    print(dataframe.shape)
else:
    print('NO duplicate data')
    

Duplicate rows found
Number of duplicate rows=  15112
Dropping duplicates

(407307, 2)


In [6]:
# download the library to for the nltk functions to use in the cleaning process
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Parameow\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Parameow\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Parameow\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import string
from sklearn import set_config

set_config(transform_output="pandas")

wnl = WordNetLemmatizer()


# Function for cleaning and tokenize the headline
def tokenize(doc):
    document = doc.lower()  # convert the content of the headline to lowercase
    document = re.sub(r'\d+', '',
                      document)  # remove all of the digits inside of the content (using regular expressions)
    document = document.translate(str.maketrans('', '', string.punctuation))  # remove the puntuations (, . ! # ...)
    document = document.strip()  # remove the spaces at the start and end of the headline
    return [wnl.lemmatize(token) for token in word_tokenize(document) if token not in stopwords.words('english')]
    # tokenize the headlines
    # and then filter only the words that are not in the english stopwords (words that are commonly used and give no benifits to the classifier)
    # and finally lemmatize all of the tokens


# The preprocess pipeline
preprocessor = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),  # passing custom tokenizer method for the CountVectorizer to use
    ('tfidf', TfidfTransformer()),
])

X = dataframe["TITLE"]
Y = dataframe["CATEGORY"]

X_train, X_test, y_train, y_test = train_test_split(
    X,
    Y,
    test_size = 0.3,
    random_state=42
)

x_tfidf_train = preprocessor.fit_transform(X_train)  # process the training dataset
x_tfidf_test = preprocessor.transform(X_test)  # process the test dataset
# tfidf_test = preprocessor.transform(X_test.values) # process the testing dataset

C:\Users\Parameow\anaconda3\envs\pythonProject\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


# Training Model

## Label encoder

In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_label_train = le.fit_transform(y_train)
y_label_test = le.fit_transform(y_test)

array([1, 3, 0, ..., 1, 0, 0])

# Decision Tree Classifier

In [9]:
dt_classifier = DecisionTreeClassifier(criterion="gini", splitter="best")
dt_classifier.fit(x_tfidf_train, y_label_train)
dt_predictions = dt_classifier.predict(x_tfidf_test)

# Evaluation
print("accuracy score of Decision Tree:")
print(accuracy_score(y_label_test, dt_predictions))

accuracy score of Decision Tree:
0.9011399998363245



# Multinomial Naive Bayes Classifier

In [10]:
nb_classifier = MultinomialNB()
nb_classifier.fit(x_tfidf_train, y_label_train)
nb_predictions = nb_classifier.predict(x_tfidf_test)

# Evaluation
print("accuracy score of Multinomial Naive Bayes:")
print(accuracy_score(y_label_test, nb_predictions))

accuracy score of Multinomial Naive Bayes:
0.9206255677493801



# Artificial Neural Network

In [11]:
nn_classifier = MLPClassifier()
nn_classifier.fit(x_tfidf_train, y_label_train)
nn_predictions = nn_classifier.predict(x_tfidf_test)

# Evaluation
print("accuracy score of Artificial Neural Network:")
print(accuracy_score(y_label_test, nn_predictions))

accuracy score of Artificial Neural Network:
0.9483849320337499
